In [35]:
import pandas as pd
rev = pd.read_csv(r"question_responce.csv")
rev.head()

,Unnamed: 0,responce,question
0,603,@115904 We'll be sure to pass along your kind ...,@AmericanAir Erica on the lax team is amazing ...
1,605,@115904 Our apologies for the delay in respond...,@AmericanAir Could you have someone on your la...
2,608,"@115905 Aww, that's definitely a future pilot ...",Ben Tennyson and an American Airlines pilot. 🎃...
3,612,@115906 We're sorry for your frustration.,"@AmericanAir Right, but I earned those. I also..."
4,618,@115909 We're glad you got to kick back and en...,"Thank you, @AmericanAir for playing #ThisIsUs ..."


In [36]:
import nltk
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
stop_words=set(nltk.corpus.stopwords.words('english'))

In [37]:
def clean_text(headline):
    le=WordNetLemmatizer()
    word_tokens=word_tokenize(headline)
    tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
    cleaned_text=" ".join(tokens)
    return cleaned_text
rev['cleaned_text']=rev['question'].apply(clean_text)

In [53]:
rev["cleaned_text"] = rev["cleaned_text"].str.replace('AmericanAir','')
rev["cleaned_text"] = rev["cleaned_text"].str.replace('http','')
print(rev.cleaned_text)

0                           Erica team amazing give raise
1            Could someone team available guide gate ASAP
2       Tennyson American Airlines pilot trunkortreat ...
3        Right earned also pas spouse need change program
4       Thank  playing ThisIsUs great flight attendant...
                              ...                        
1847     nailed transatlantic WiFi service able join 1...
1848                            Average price ticket 2500
1849     Really annoyed month since damaged claim neve...
1850     terrible service wait age trying call number ...
1851     charge patron change flight every time airpor...
Name: cleaned_text, Length: 1852, dtype: object


In [54]:
# Building a dictionary of responses

vect =TfidfVectorizer(stop_words=stop_words,max_features=1000)
vect_text=vect.fit_transform(rev['cleaned_text'])
print(vect_text)

  (0, 412)	0.5529514529306722
  (0, 54)	0.5939070649598848
  (0, 879)	0.5843963457214582
  (1, 78)	0.5225608089087591
  (1, 407)	0.29135401046590886
  (1, 90)	0.42532865299362604
  (1, 826)	0.3951006690273464
  (1, 219)	0.3644699121367777
  (1, 879)	0.4149647250090478
  (2, 185)	0.3876113606433001
  (2, 56)	0.33519824012199595
  (2, 11)	0.4396722834272028
  (2, 431)	0.4592973392656634
  (2, 690)	0.310682514221521
  (2, 42)	0.36472730223257405
  (2, 55)	0.321844963626745
  (3, 718)	0.426452435408527
  (3, 154)	0.3242530242433215
  (3, 625)	0.28355004236601444
  (3, 671)	0.4016683695772027
  (3, 52)	0.36034610238509834
  (3, 293)	0.4660683225748587
  (3, 770)	0.352407712950135
  (4, 463)	0.41539759593853104
  (4, 97)	0.40306907044082546
  :	:
  (1849, 814)	0.28733652195466486
  (1849, 175)	0.32202482108375513
  (1849, 280)	0.2975232847332916
  (1849, 627)	0.25783089707831724
  (1849, 614)	0.2975232847332916
  (1849, 733)	0.24923233474029236
  (1849, 905)	0.26649614912731695
  (1849, 97)	

In [55]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=20,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(vect_text)

In [56]:
print(lda_top[0])

[0.0183066  0.0183066  0.0183066  0.01830662 0.65217451 0.0183066
 0.0183066  0.0183066  0.0183066  0.0183066  0.0183066  0.0183066
 0.0183066  0.0183066  0.0183066  0.0183066  0.0183066  0.0183066
 0.0183066  0.0183066 ]


In [57]:
print("Document 1: ")
for i,topic in enumerate(lda_top[2]):
  print("Topic ",i,": ",topic*100,"%")

Document 1: 
Topic  0 :  1.3815841480660318 %
Topic  1 :  39.759770851935556 %
Topic  2 :  1.3815841453207605 %
Topic  3 :  1.3815841550868397 %
Topic  4 :  1.3815841565226727 %
Topic  5 :  1.381584145311549 %
Topic  6 :  1.3815841477191166 %
Topic  7 :  1.3815841939776825 %
Topic  8 :  1.3815841461219234 %
Topic  9 :  1.3815841452514457 %
Topic  10 :  1.381584147758489 %
Topic  11 :  1.3815841454612399 %
Topic  12 :  1.3815841464331735 %
Topic  13 :  1.3815841452863147 %
Topic  14 :  1.3815841466234657 %
Topic  15 :  1.3815841491606866 %
Topic  16 :  1.3815841745813682 %
Topic  17 :  35.371714417693795 %
Topic  18 :  1.381584145212333 %
Topic  19 :  1.3815841464755567 %


In [58]:
vocab = vect.get_feature_names()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
     print("\nTopic "+str(i)+": ")
     for t in sorted_words:
            print(t[0],end=" ")


Topic 0: 
checked luggage fantastic stay turkey kudos yall error lame standard 
Topic 1: 
co sent flight airport mile delayed american today time month 
Topic 2: 
much locator record transit screen mail route laptop economy basic 
Topic 3: 
flight thank great service co time please help gate home 
Topic 4: 
upgrade class check first lounge boarding co using people flight 
Topic 5: 
child nonsense drink young really disappointed airplane level economy parent 
Topic 6: 
good look flight start attendant made amazing thanks co worked 
Topic 7: 
flying never year co time refund awesome yesterday airline every 
Topic 8: 
price response suitcase detail frequent status flyer kept public telling 
Topic 9: 
hell entertainment purchased aircraft mile earned counting aadvantage statement card 
Topic 10: 
gate plane wifi delayed broken agent number late flight wish 
Topic 11: 
info fact showing idea meal literally calling upgrade list evening 
Topic 12: 
found lost switch dreamliner complete husba